In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import time
import os

os.environ[
    "PYSPARK_SUBMIT_ARGS"
] = "--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell"


APP_NAME = os.getenv("APP_NAME", "spark-streaming-app")
MASTER = os.getenv("MASTER", "local")

spark = SparkSession.builder.appName("APP_NAME").master(MASTER).getOrCreate()
spark


:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-593f2412-411c-496f-886e-e389afde0543;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in 

In [17]:
df = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka-cluster-kafka-bootstrap:9092")
    .option("subscribe", "app_messages")
    .option("startingOffsets", "latest")
    .load()
)

In [18]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [19]:
from pyspark.sql.types import StructType, StructField, FloatType, StringType
from pyspark.sql.functions import col, from_json

# {
#     "request_id": "44d4a680-b67d-4375-b27c-d07afe302d81", 
#     "data": {
#         "CPU": 36.23,
#         "memory": 26.89,
#         "disk": 38.94
#     }
# }

schema = StructType([
    StructField('request_id', StringType(), True),
    StructField('data', StructType([
        StructField('CPU', FloatType(), True),
        StructField('memory', FloatType(), True),
        StructField('disk', FloatType(), True)
    ])),
])

In [20]:
# schema = StructType() \    
#     .add("CPU", StringType(), True) \
#     .add("memory", StringType(), True) \
#     .add("disk", StringType(), True)

df = df.selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", schema).alias("data")) \
    .select("data.*")

df.printSchema()

root
 |-- request_id: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- CPU: float (nullable = true)
 |    |-- memory: float (nullable = true)
 |    |-- disk: float (nullable = true)



In [21]:
df = df.select('request_id', \
    F.col('data').getItem('CPU').alias('CPU'), \
    F.col('data').getItem('memory').alias('memory'), \
    F.col('data').getItem('disk').alias('disk'))

df.printSchema()

root
 |-- request_id: string (nullable = true)
 |-- CPU: float (nullable = true)
 |-- memory: float (nullable = true)
 |-- disk: float (nullable = true)



In [22]:
# Start streaming on console

# df.writeStream \
#       .format("console") \
#       .start() \
#       .awaitTermination()

query = df \
    .writeStream \
    .format("console") \
    .start()

time.sleep(10) # sleep 10 seconds

query.stop()

22/05/14 20:06:54 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1c04ec30-ed33-40f6-8296-82cc8f11e12b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/14 20:06:54 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----------+---+------+----+
|request_id|CPU|memory|disk|
+----------+---+------+----+
+----------+---+------+----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+------+----+
|          request_id|  CPU|memory|disk|
+--------------------+-----+------+----+
|63e0fcfd-a412-4d2...|17.55| 66.31|99.1|
+--------------------+-----+------+----+



In [ ]:
stop

In [ ]:
import time

df = df.selectExpr("CAST(value AS STRING)").selectExpr("CAST(value AS STRING)")

# Start running the query that prints the running counts to the console
query = df \
    .writeStream \
    .format("console") \
    .start()

# # query.awaitTermination()

time.sleep(10) # sleep 10 seconds

query.stop()

In [ ]:
query = values.writeStream \
    .format("json") \
    .outputMode("append")
    .start("./topic.json")
    
import time

time.sleep(10) # sleep 10 seconds

query.stop()

In [ ]:
df.writeStream \
      .format("console") \
      .outputMode("append") \
      .start() \
      .awaitTermination()

In [ ]:
spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka-cluster-kafka-bootstrap:9092") \
    .option("subscribe", "app_messages") \
    .option("startingOffsets", "latest") \
    .load() \
    .writeStream \
    .format("json") \
    .outputMode("append") \ # .option("checkpointLocation", bronze_checkpoint_loc_vehicle) \
    .start("topic.json")